<center><font size="20">Flask Basics</font></center>

# Imports

In [1]:
from flask import Flask

app = Flask(__name__)

# Definitons

* <code>\__name__</code>: Refers to the current local file. If executed its equal to <code>"__main__"</code>, if imported equals to the **file name**.<br></br>

* <code>@app.route("/")</code>: function decorator that is called *routes* or *wheels*. Which URL the code is navigationg through and display some HTML code. Argument inside <code>@app.route("/")</code> refers to webpage one is working on. <code>"/"</code> refs to the home page or root URLof the website.<br></br>



# Flask
## Routes
* Routes are webpages. 
* Each route corresponds to a separate webpage. 
* It has to have a reference to the app (via <code>@app.route(''))</code> decorator. and have designated extension to the <code>root</code> URL or just forwardslash <code>'/'</code> for the root/ home page of the website. 
* Each route can have multiple extensions:
<code>
    @app.route("/")
    @app.route("/home")
    def home_page():
        return render_template('home.html')
</code>

In [2]:
@app.route("/") # referes to the home page of the webpage
def hello_world() -> str:
    return "<h1>Hello, World!</h1>"

@app.route("/about") # referes to the about page 
def about_page() -> str:
    return "<h1>About Page</h1>"

##### Reference same route multiple ways
Adding several route decorators we can reference the same page via different aliases or forward slash addresses. <code>"/home"</code> parameter will reference to the home page of the web app.

In [3]:
@app.route("/")
@app.route("/home")
def home_page():
    return render_template('home.html')

### Dynamic Routes

Creates generic wepages per argument

In [4]:
@app.route("/about/<username>")
def about_page2(username: str) -> str:
    return f"<h1>This is the about page of {username}</h1>"

## Database

### Create Database
When using <code>sqlite</code> with **Unix** machines, four <code>////</code> are required as well as absoulte path to the project file.

<code>app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{Path.cwd()}/market.db'
db = SQLAlchemy(app) # database
</code>

To create the database file (to include the specified table(s)) use terminal when <code>flask</code> is not running and make sure to be in the projects root director. Type in the terminal:

<code>flask shell</code><br>
<code>db.create_all()</code><br>

To bypass the context RuntimeError. 

### Add Data
Need to use <code>flask shell</code> in the terminal.

<code>
    item1 = Item(name='IPhone 10', price=500, barcode='123456789011', description='desc')
    db.session.add(item1)
    db.session.commit()
</code>

To query the elements in the DB write the following:

<code>
    Item.query.all()
</code>

## Validations

### Init Validation Class
* Create an object (class) to be validated.
* Inherit from FlaskForm base class
* Import neccessary classes for different types of data
* Import library classes to validate differnet data types

Python file **"forms.py"** where the form to-be-filled by user is defined:
* <code>FlaskForm</code> Base form to be inherited from.
* <code>DataRequired</code>: Validates if corresponding data is entered, only whitespaces are considered empty.
* <code>PasswordField</code> Makes sure the input is not visible on the screen.
* <code>EqualTo</code> Makes sure that entry equals with referred field. For confirmation form fields usually.

In [5]:
# 3rd party imports
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import Length, EqualTo, Email, DataRequired


class RegisteredForm(FlaskForm):
    
    username = StringField(
        label='User Name:', 
        validators=[Length(min=2, max=30), DataRequired()])
    
    email = StringField(
        label='Email Address:',
        validators=[Email(), DataRequired()])
    
    password1 = PasswordField(
        label='Password:',
        validators=[Length(min=6), DataRequired()])
    
    password2 = PasswordField(
        label='Confirm Password:',
        validators=[EqualTo('password1'), DataRequired()])
    
    submit = SubmitField(label='Create Account')

### Route Methods

To validate that some information is input correctly a **'POST'** HTTP method has to be used for a <code>route</code> so data from HTML can be fetched. Refer to different methods below: 

In [8]:
import pandas as pd
import requests

url = requests.get('https://pythonbasics.org/flask-http-methods/')

methods_df = pd.read_html(io=url.text)[0]

print(methods_df.to_string())

  Request                                                                                                       Purpose
0     GET                                    The most common method. A GET message is send, and the server returns data
1    POST  Used to send HTML form data to the server. The data received by the POST method is not cached by the server.
2    HEAD                                                                     Same as GET method, but no response body.
3     PUT                             Replace all current representations of the target resource with uploaded content.
4  DELETE                                  Deletes all current representations of the target resource given by the URL.


In [11]:
@app.route('/register', methods=['GET', 'POST'])
def register_page():
    form = RegisteredForm()
    return render_template('register.html', form=form)

### Submit & Redirect
Line 15

In [12]:
app.route('/register', methods=['GET', 'POST'])
def register_page2():
    form = RegisteredForm()
    
    # submit button validation and add created data to database
    if form.validate_on_submit():
        user_to_create = User(username=form.username.data,
                              email=form.email.data,
                              password_hash=form.password1.data)
        # send data to database
        db.session.add(user_to_create)
        db.session.commit()
        
        # redirect person to new page
        return redirect(url_for('market_page'))
    
    return render_template('register.html', form=form)

### Error Handling
Line 19 - 22.

In [2]:
# routes.py
@app.route('/register', methods=['GET', 'POST'])
def register_page3():
    # inits the form class from "forms.py"
    form = RegisteredForm()
    
    # submit button validation and add created data to database
    if form.validate_on_submit():
        user_to_create = User(username=form.username.data,
                              email=form.email.data,
                              password_hash=form.password1.data)
        # send data to database
        db.session.add(user_to_create)
        db.session.commit()
        
        # redirect person to new page
        return redirect(url_for('market_page'))
    
    # if errors occur they are sent to forms.errors
    if form.errors != {}: # if errors dict is not empty
        for err_msg in form.errors.values():
            print(f'There was an error with creating a user: {err_msg}')
    
    
    return render_template('register.html', form=form)

#### Flashed Error Messages
* <code>flash()</code>: To capture errors to be flashed for the user in "routes.py" file.

In [3]:
@app.route('/register', methods=['GET', 'POST'])
def register_page4():
    form = RegisteredForm()
    # ...
    
    # if errors occur they are sent to forms.errors
    if form.errors != {}: # if tehre are errors
        for err_msg in form.errors.values():
            flash(f'There was an error with creating a user: {err_msg}', category='danger')
    
    
    return render_template('register.html', form=form)

* <code>get_flashed_messages()</code> To nicely output the messages for the user. Embed in the <code>*.html</code> file, usually the base file. <code>with_categories</code> an Optional Parameter tuple to mention the category(error/info/warning).

In [ ]:
%%html
<body>
      <nav class="navbar navbar-expand-md navbar-dark bg-dark">
      </nav>
            <!-- Use context manager -->
            {% with messages = get_flashed_messages(with_categories=true) %}
                {% if messages %}
                    {% for category, message in messages %}
                        <div class="alert alert-{{ category }}">
                            <button type="button" class="m1-2 mb-1 close" data-dismiss="alert" aria-label="Close">
                                <span aria-hidden="true">&times;</span>
                            </button>
                            {{ message }}
                        </div>
                    {% endfor %}
                {% endif %}
            {% endwith %}
</body>

#### Existing Usernames
To validate that there are no duplicate usernames, emails etc in the database we can add validation functions for such fields in the **forms.py** file. The way <code>FlaskForm</code> validates and outputs errors works as follows:
* FlaskForm checks for prefix <code>validate_</code> and postfix <code>field_name</code> in the function name to know to raise an error if specified field (username) duplicate is enetered.

In [8]:
# forms.py

# 3rd party imports
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import Length, EqualTo, Email, DataRequired, ValidationError


class RegisteredForm(FlaskForm):
    
    # FlaskForm will validate the username via 'validate_' 'username' function name.
    # flask validates if it sees prefix 'validate_' in function name and looks for 
    # field 'username' in the User database.
    def validate_username(self, username_to_check: str):
        user = User.query.filter_by(username=username_to_check.data).first()
        if user:
            raise ValidationError('Username already exists! Please try a different username.')
    
    def validate_email(self, email_to_check: str):
        email = User.query.filter_by(email=email_to_check.data).first()
        if email:
            raise ValidationError('Email address already exists! Please try a different email address.')
        
    
    username = StringField(
        label='User Name:', 
        validators=[Length(min=2, max=30), DataRequired()])
    
    email = StringField(
        label='Email Address:',
        validators=[Email(), DataRequired()])


## Autenthication

### Encrypt Password
1. Create encryption class <code>Bcrypt(app)</code> instance from <code>flask_bcrypt</code> package in the <code>__init__.py</code> file.

In [6]:
# __init__.py
from flask_bcrypt import Bcrypt
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
db = SQLAlchemy(app)

# store passwords as hashes not plain text
bcrypt = Bcrypt(app)

ModuleNotFoundError: No module named 'flask_sqlalchemy'

2. In the <code>models.py</code> file or where database table is defined add a new (password) class <code>@property</code> by returning it to <code>self</code>.
3. After that, overwrite defiend class attribute <code>password_hash</code> with the <code>@password.setter</code> decorator and encrypt the password using <code>bcrypt</code> instance, it's ash decoded to 'utf-8 format:  

In [4]:
class User(db.Model):
    
    # 'db.relationship() creates relationla link between Item & User tables
    id = db.Column(db.Integer(), primary_key=True)
    username = db.Column(db.String(length=30), nullable=False, unique=True)
    email = db.Column(db.String(length=50), nullable=False, unique=True)
    password_hash = db.Column(db.String(length=60), nullable=False)
    budget = db.Column(db.Integer(), nullable=False, default=1000)
    items = db.relationship('Item', backref='owned_user', lazy=True)


    @property
    def password(self):
        return self.password

    @password.setter
    def password(self, plain_text_password):
        self.password_hash = bcrypt.generate_password_hash(plain_text_password).decode('utf-8')

NameError: name 'db' is not defined

4. Finally, we have to set or change the class/table <code>User</code> parameter in the <code>routes.py</code> file to <code>password</code> which we defined as **User** method that embeds <code>password_hash</code> encryption protocol.

In [7]:
# routes.py
@app.route('/register', methods=['GET', 'POST'])
def register_page():
    form = RegisteredForm()
    
    # submit button validation and add created data to database
    if form.validate_on_submit():
        user_to_create = User(username=form.username.data,
                              email=form.email.data,
                              
                              # change this field to password, not password_hash
                              password=form.password1.data)

### Log In
1. Initialize <code>login_manager</code> in <code>__init__.py</code> file

In [25]:
%%script false --no-raise-error
# __init__.py
from flask_login import LoginManager

# ...

# init login manager
login_manager = LoginManager(app)

2. Create a login form in <code>forms.py</code>

In [19]:
# forms.py
from wtforms import StringField, PasswordField, SubmitField

class LoginForm(FlaskForm):
    
    username = StringField(label='User Name:', validators=[DataRequired()])
    password = PasswordField(label='Password:', validators=[DataRequired()])
    submit = SubmitField(label='Sign In')

2. Add a load_user callback to <code>models.py</code> which keeps the user logged when they browse the webpage.
3. Import <code>UserMixin</code> fom <code>flask_login</code> to provide <code>User</code> class functionality to keep user logged in when done successfully.
4. Add User method <code>check_password_correction</code> to check if entered password matches with the password hash in the db 

In [26]:
%%script false --no-raise-error
# local imports
from market import db, bcrypt, login_manager

# 3rd party imports
from flask_login import UserMixin

# callback for the web app. If user is logged in and refreshes or browses through
# the webpage the app keeps them logged in, therefore using different session route 
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    #...
    
    # User class method to check input password hash matches hash in db
    def check_password_correction(self, attempted_password) -> bool:
        return bcrypt.check_password_hash(self.password_hash, attempted_password)

5. Create a login page in <code>routes.py</code>
6. Use <code>['GET', 'POST']</code> request methods to communicate with the input data.
7. Init <code>LoginForm()</code>
8. Init and query entered username:<br> 
<code>attempted_user = User.query.filter_by(username=form.username.data).first()</code>
9. Check if user exists and entered password matches with the one in db
10. If so, login the user <code>login_user</code> and prompt success messages and redirect to market page by default.

In [28]:
%%script false --no-raise-error
# routes.py

# local imports
from market.forms import LoginForm

# 3rd party imports
from flask_login import login_user

@app.route('/login', methods=['GET', 'POST'])
def login_page():
    form = LoginForm()
    if form.validate_on_submit():
        # if user excists. first() is used to get the object from query
        attempted_user = User.query.filter_by(username=form.username.data).first()
        if attempted_user and attempted_user.check_password_correction(
            attempted_password=form.password.data
        ):
            # when entered username and password are correct log in
            login_user(attempted_user)
            flash(f'Success! You are logged in as {attempted_user.username}', category='success')
            
            return redirect(url_for('market_page'))
        
        else:
            flash('Username and password do not match! Please try again!', category='danger')
        
    return render_template('login.html', form=form)

### Log Out
1. Create logout route in the <code>routes.py</code> file, with confirmation message in blue as specified by the <code>category='info'</code> argument in the <code>flash()</code> function.

In [6]:
%%script false --no-raise-error
from flask_login import logout_user,

@app.route('/logout')
def logout_page():
    logout_user()
    flash('Your account has successfully logged out!', category='info')
    return redirect(url_for('home_page'))

2. Direct user to <code>logout_page</code> when clicked on the **Logout** link/button in the <code>*.html</code> file with <code>{{ url_for('logout_page') }}</code>.

In [7]:
%%script false --no-raise-error
<body>
    <div class="collapse navbar-collapse" id="navbarNav">
        {% if current_user.is_authenticated %}
            <a class="nav-link" href="{{ url_for('logout_page') }}">Logout</a>
        {% else %}
            <!-- ... -->
        {% endif %}
    </div>
</body>

### Login Recuired
To make certain web-pages accessible only when logged in:
1. Add <code>login_required</code> function as decorator to the needed <code>route</code> in the <code>routes.py</code> file

In [8]:
%%script false --no-raise-error
from flask_login import login_required

@app.route('/market')
@login_required
def market_page():
    items = Item.query.all()
    return render_template('market.html', items=items)

### Login right after Register
To login user right after registration is complete:
1. add <code>login_user(user_to_create)</code> line to the <code>register_page()</code> route after data has sent to the db. <code>login_user(User)</code> function handles logging in the user.
2. Display informative message after successful login using <code>flash(msg, category='success')</code> function

In [9]:
%%script false --no-raise-error
# routes.py

@app.route('/register', methods=['GET', 'POST'])
def register_page():
    form = RegisteredForm()
    
    # when submit is pressed it validates if no errors occured
    if form.validate_on_submit():
        user_to_create = User(username=form.username.data,
                              email=form.email.data,
                              password=form.password1.data)
        # send data to database
        db.session.add(user_to_create)
        db.session.commit()
        
        # when entered username and password are correct log in
        login_user(user_to_create)
        flash(f"Account created successfully! You are now logged in as {user_to_create.username}.", category='success')
        
        # redirect person to new page
        return redirect(url_for('market_page'))

# Jinja2 Syntax
## Print Variables
We specify the <code>item_name</code> argument in the <code>.py</code> file and add <code>{{ }}</code> bracets to the <code>templates/foo.html</code> file.

In [5]:
@app.route('/market')
def market_page():
    return render_template('market.html', item_name='Phone')

<code><p>{{ item_name }}</p></code>

In [6]:
%%html
<body>
    <h1>Market Page</h1>
    <p>{{ item_name }}</p>
    
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous"></script>
</body>

### For Loop

In [7]:
items = [
        {'id': 1, 'name': 'Phone', 'barcode': '893212299897', 'price': 500},
        {'id': 2, 'name': 'Laptop', 'barcode': '123985473165', 'price': 900},
        {'id': 3, 'name': 'Keyboard', 'barcode': '231985128446', 'price': 150}
    ]

<code>
    {% for item in items %}
        <tr>
            <td>{{ item.id }}</td>
            <td>{{ item.name }}</td>
            <td>{{ item.barcode }}</td>
            <td>${{ item.price }}</td>
        </tr>
    {% endfor %}
</code>

In [8]:
%%html
<table class="table table-hover table-dark">
        <thead>
            <tr>
                <th scope="col">ID</th>
                <th scope="col">Name</th>
                <th scope="col">Barcode</th>
                <th sc<td>${{ item.price }}</td>ope="col">Price</th>
            </tr>
        </thead>
        <tbody>
             {% for item in items %}
                    <tr>
                        <td>{{ item.id }}</td>
                        <td>{{ item.name }}</td>
                        <td>{{ item.barcode }}</td>
                        <td>${{ item.price }}</td>
                    </tr>
                {% endfor %}
        </tbody>
    </table>

ID,Name,Barcode,"${{ item.price }}ope=""col"">Price"
{{ item.id }},{{ item.name }},{{ item.barcode }},${{ item.price }}


### Template Inheritage
* Using a template inheritage so that some parts of the pages of a webpage are generic (like navigation part, header, footer etc) yet some parts are unique per page. 
* Use special <code>jinja</code> syntax in <code>{%  %}</code> brackets in the <code>base.html</code> file. 

Snippet of the <code>base.html</code> file:

In [9]:
%%html
<body>
      <!-- Navbar here -->
      <nav class="navbar navbar-expand-md navbar-dark bg-dark">
        <a class="navbar-brand" href="#">Flask Market</a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNav">
          <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navbarNav">
            <ul class="navbar-nav mr-auto">
                <li class="nav-item active">
                    <a class="nav-link" href="{{ url_for('home_page') }}">Home <span class="sr-only">(current)</span></a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" href="{{ url_for('market_page') }}">Market</a>
                </li>
            </ul>
            <ul class="navbar-nav">
                <li class="nav-item">
                    <a class="nav-link" href="#">Login</a>
                </li>
                <li class="nav-item">
                    <a class="nav-link" href="{{ url_for('register_page') }}">Register</a>
                </li>
            </ul>
        </div>
    </nav>
</body>

Snippet of <code>*.html</code> file that inherits from base:

In [10]:
%%html
{% extends 'base.html' %}

{% block title %}
    Registration Page
{% endblock %}


### Navigation Logic
Need to change the <code>href</code> CSS tag programatically with jinja syntax and function <code>url_for("route_name")</code> 

In [11]:
%%html
<div class="collapse navbar-collapse" id="navbarNav">
    <ul class="navbar-nav mr-auto">
        <li class="nav-item active">
            <a class="nav-link" href="{{ url_for('home_page') }}">Home <span class="sr-only">(current)</span></a>
        </li>
        <li class="nav-item">
            <a class="nav-link" href="{{ url_for('market_page') }}">Market</a>
        </li>
    </ul>
    <ul class="navbar-nav">
        <li class="nav-item">
            <a class="nav-link" href="#">Login</a>
        </li>
        <li class="nav-item">
            <a class="nav-link" href="#">Register</a>
        </li>
    </ul>
</div>

## Register Page

<code>forms.py</code>file

In [8]:
# 3rd party imports
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import Length, EqualTo, Email, DataRequired


class RegisteredForm(FlaskForm):
    
    username = StringField(
        label='User Name:', 
        validators=[Length(min=2, max=30), DataRequired()])
    
    email = StringField(
        label='Email Address:',
        validators=[Email(), DataRequired()])
    
    password1 = PasswordField(
        label='Password:',
        validators=[Length(min=6), DataRequired()])
    
    password2 = PasswordField(
        label='Confirm Password:',
        validators=[EqualTo('password1'), DataRequired()])
    
    submit = SubmitField(label='Create Account')

<code>*.html</code> file

In [6]:
%%html
{% extends 'base.html' %}

{% block title %}
    Registration Page
{% endblock %}

{% block content %}
    <body class="text-center">
        <div class="container"  style="width: 600px">
            <form method="POST" class="form-register" style="color:white">
                {{ form.hidden_tag() }} <!-- To prevent CSRF attack on db -->
                <img class="mb-4", src="https://imgs.search.brave.com/YPuYCl5-EmeqfaL-FzqF8l4egROpKnuuhEcQJWq3eUY/rs:fit:180:180:1/g:ce/aHR0cDovL2Jpb3dl/Yi51d2xheC5lZHUv/YmlvMjAzL3MyMDA4/L2Fybm9sZF9tZWFn/L01DajA0MzQ3Mjkw/MDAwWzFdLnBuZw" 
                    style="width:100px;height:100px;">
                <h1 class="h3 mb-3 font-weight-normal">
                    Please Create your Account
                </h1>
                <br>
                {{ form.username.label() }}
                {{ form.username(class="form-control", placeholder="User Name") }}

                {{ form.email.label() }}
                {{ form.email(class="form-control", placeholder="Email Address") }}

                {{ form.password1.label() }}
                {{ form.password1(class="form-control", placeholder="Password") }}

                {{ form.password2.label() }}
                {{ form.password2(class="form-control", placeholder="Confirm Password") }}
                
                <br>

                {{ form.submit(class='btn btn-lg btn-block btn-primary') }}

            </form>
        </div>
    </body>
{% endblock %}

## Login Page

In [11]:
# forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired

class LoginForm(FlaskForm):
    username = StringField(label='User Name:', validators=[DataRequired()])
    password = PasswordField(label='Password:', validators=[DataRequired()])
    submit = SubmitField(label='Sign In')

In [12]:
# routes.py
#from market.forms import LoginForm
from flask import render_template

@app.route('/login', methods=['GET', 'POST'])
def login_page():
    form = LoginForm()
    return render_template('login.html', form=form)

In [15]:
%%html
<!--{% extends 'base.html' %}

{% block title %}
    Login Page
{% endblock %}

{% block content %}-->

<body class="text-center">
    <div class="container" style="width: 600px">
        <form method="POST" class="form-signin" style="color:white">
            {{ form.hidden_tag() }}
            <img class="mb-4", src="https://imgs.search.brave.com/YPuYCl5-EmeqfaL-FzqF8l4egROpKnuuhEcQJWq3eUY/rs:fit:180:180:1/g:ce/aHR0cDovL2Jpb3dl/Yi51d2xheC5lZHUv/YmlvMjAzL3MyMDA4/L2Fybm9sZF9tZWFn/L01DajA0MzQ3Mjkw/MDAwWzFdLnBuZw" 
                    style="width:100px;height:100px;">
            <h1 class="h3 mb-3 font-weight-normal">
                Please Login
            </h1>
            <br>
            {{ form.username.label() }}
            {{ form.username(class='form-control', placeholder='User Name') }}

            {{ form.password.label() }}
            {{ form.password(class='form-control', placeholder='Password') }}

            <br>



            <div class="checkbox mb-3">
                <h6>Do not have an account?</h6>
                <a class="btn btn-sm btn-secondary" href="{{ url_for('register_page') }}">Register</a>
            </div>

            {{ form.submit(class='btn btn-lg btn-block btn-primary') }}

        </form>
    </div>
</body>

<!--{% endblock %}-->

## Fontawesome Icons
To easily integrate <code>fontawesome</code> icons to the webpage use <code>flask_fontawesome</code> package with the following setup:
1. Create <code>FontAwesome</code> object in the <code>__init__.py</code> file

In [2]:
%%script false --no-raise-error
from flask_fontawesome import FontAwesome

app = Flask(__name__)
fa = FontAwesome(app)

Modify respective <code>*.html</code> file:
2. Add line <code>{{ fontawesome_html() }}</code> to the <code>head</code> of the html.
3. Insert desired icon using <code>class="fas fa-coins"</code> class:

In [5]:
%%script false --no-raise-error
<!doctype html>
<html lang="en">
<head>
  {{ fontawesome_html() }}
</head>
<body>
    <p class="nav-item">
        <a class="nav-link" style="color: lawngreen; font-weight: bold">
            <i class="fas fa-coins"></i>
        </a>
    </p>
</body>

# HTML Syntax

## div
Setting the size of the box on the webpage

In [9]:
%%html
<div class="container">Yee</div>

## Table
### Create Table

<code>'id'</code> is the convention in Flask apps No missing values are allowed with <code>nullable</code> and name has to be unique in the table.

<code>
    from flask_sqlalchemy import SQLAlchemy
    
    class Item(db.Model):
        id = db.Column(db.Integer(), primary_key=True)
        name = db.Column(db.String(length=30), nullable=False, unique=True) 
        price = db.Column(db.Integer(), nullable=False)
        barcode = db.Column(db.String(length=12), nullable=False, unique=True)
        description = db.Column(db.String(length=1024), nullable=False, unique=True)</code>

### Bootstrap Table Template

In [12]:
%%html
<table class="table table-hover table-dark">
        <thead>
            <tr>
                <!-- Your Columns HERE -->
                <th scope="col">ID</th>
                <th scope="col">Name</th>
                <th scope="col">Barcode</th>
                <th scope="col">Price</th>
            </tr>
        </thead>
        <tbody>
            <!-- Your rows inside the table HERE: -->
                <tr>
                    <td>Value for Id</td>
                    <td>Value for Name</td>
                    <td>Value for Barcode</td>
                    <td>Value for Price</td>
                </tr>
        </tbody>
    </table>

ID,Name,Barcode,Price
Value for Id,Value for Name,Value for Barcode,Value for Price


## Button

In [13]:
%%html
<h1>Market Page</h1>
    <table class="table table-hover table-dark">
        <thead>
            <tr>
                <!-- Your Columns HERE -->
                <th scope="col">Name</th>
                <th scope="col">Barcode</th>
                <th scope="col">Price</th>
                <th scope="col">Options</th>
            </tr>
        </thead>
        <tbody>
            <!-- Your rows inside the table HERE: -->%%html
                {% for item in items %}
                    <tr>
                        <td>{{ item.name }}</td>
                        <td>{{ item.barcode }}</td>
                        <td>${{ item.price }}</td> <!-- '$' adds dollar symbol -->
                        <td>
                            <button class="btn btn-outline btn-info">More Info</button>
                            <button class="btn btn-outline btn-success">Purchase this Item</button>
                        </td>
                    </tr>
                {% endfor %}
        </tbody>
    </table>

Name,Barcode,Price,Options
{{ item.name }},{{ item.barcode }},${{ item.price }},More Info Purchase this Item


## "X" to Close Window

In [6]:
%%html
<span aria-hidden="true">&times;</span>

## Modals

### 'GET' method
Modals are pop-up message windows that are prompted when a button, link or after action. Modals are usually implemented in separate directory in <code>templates</code> directory.
1. Create a directory <code>includes</code> in the <code>templates</code> directory.
2. Create a html code for the pop-up window aka <code>modal</code>
3. If unique modals are required then modal <code>id</code> has to be parametrized using <code>jinja</code> syntax.

In [11]:
%%script false --no-raise-error

<!-- 
templates/includes/items_modals.html
Modal for More Info button 
-->

<div class="modal fade" id="Modal-MoreInfo-{{ item.id }}"
     tabindex="-1"
     aria-labelledby="exampleModalLabel" aria-hidden="true">
  <div class="modal-dialog">
    <div class="modal-content">
      <div class="modal-header">
        <h5 class="modal-title" id="ModalLabel">
          <!-- -->
          {{ item.name }}
        </h5>
        <button type="button" class="close"
                data-dismiss="modal" aria-label="Close">
          <span aria-hidden="true">&times;</span>
        </button>
      </div>
      <div class="modal-body">
        {{ item.description }}
      </div>
      <div class="modal-footer">
        <button type="button" class="btn btn-secondary"
                data-dismiss="modal">Close
        </button>
      </div>
    </div>
  </div>
</div>

4. Include defined modal html in desired html file using <code>jinja</code> syntax
5. Example given in a <code>for</code> loop in a data table where unique modal <code>id</code> is required per row in the table.

In [12]:
%%script false --no-raise-error

<!--
templates/market.html
-->

<tbody>
    {% for item in items %} <!-- Generate row per unique item: -->
    {% include 'includes/items_modals.html' %} <!-- Import items_modals.html per item -->
        <tr>
            <td>{{ item.id }}</td>
            <td>{{ item.name }}</td>
            <td>{{ item.barcode }}</td>
            <td>${{ item.price }}</td>
            <td>
                <button class="btn btn-outline btn-info" data-toggle="modal" data-target="#Modal-MoreInfo-{{ item.id }}">
                    More Info
                </button>
                <button class="btn btn-outline btn-success" data-toggle="modal" data-target="#Modal-PurchaseConfirm-{{ item.id }}">
                    Purchase this Item
                </button>
            </td>
        </tr>
    {% endfor %}
</tbody>

### 'POST' method
<code>form.method = 'POST'</code> means that the *User* interaction sends data to the host, which usually means information is sent via filling a form or pressing a button. In this example it is going to be pressing a button.

1. Create *Flask* form <code>class</code> object inheriting from <code>FlaskForm</code> in the <code>forms.py</code> file.

In [13]:
from flask_wtf import FlaskForm
from wtforms import SubmitField

class PurchaseItemForm(FlaskForm):
    submit = SubmitField(label='Confirm')

2. Create/modify the webpage route function by adding <code>'POST'</code> route method.
3. Init the form object <code>PurchaseItemForm</code>
4. Access Flask <code>request</code> object for <code>'POST'</code> method.
5. Grab the purchased item name <code>purchased_item</code> via hidden <code>html</code> <code>input</code> tag
6. Query item data using <code>purchased_item</code>
7. Check if such item exists and if user has enough funds with <code>can_purchase()</code> method.
8. Buy the item using <code>p_item_object</code> method.

In [14]:
%%script false --no-raise-error
@app.route('/market', methods=['GET', 'POST'])
@login_required
def market_page():
    purchase_form = PurchaseItemForm()
    
    # if data is sent from HTML to the server (aka the "POST" method)
    if request.method == "POST":
        
        # since we added hidden <input> tag to our form in items_modals.html
        # we can catch the value="{{ item.name }}" of the 'purchased_id' item
        purchased_item = request.form.get('purchased_item')
        
        # now we have the name of the item and query its data from the db
        p_item_object = Item.query.filter_by(name=purchased_item).first()
        
        if p_item_object: 
            if current_user.can_purchase(item_object=p_item_object):
                p_item_object.buy(user=current_user)
                flash(f"Congratulations! You purchased {purchased_item} for ${p_item_object.price}.", category='success')
            else:
                flash(f"Unfortunately, you don't have enough funds to purchase {p_item_object.name}.", category='danger')
            
        # end the 'POST' request with return statement
        return redirect(url_for('market_page'))
            
    if request.method == "GET":
        # only display items that have no owners, aka bought items will disappear per user
        items = Item.query.filter_by(owner=None)
        
        return render_template('market.html', items=items, purchase_form=purchase_form)

In [16]:
%%script false --no-raise-error

# models.py
class User(db.Model, UserMixin):
    # ...
    def can_purchase(self, item_object):
        return self.budget >= item_object.price
    
class Item(db.Model):
    # ...
    def buy(self, user):
        # current_user built-in method accesses logged-in user User object.
        # assigning user id to the Items owner field
        self.owner = user.id
        
        # deduct price of the item from user's budegt
        user.budget -= self.price

        db.session.commit()

In [17]:
%%script false --no-raise-error
# templates/includes/items_models.html
# ...
<form method="POST">
    {{ purchase_form.hidden_tag() }}

    <h4 class="text-center">
        Confirm purchasing {{ item.name }} for ${{ item.price}}.
    </h4>
    <br>
    <br>
    <h6 class="text-center">
        By clicking Confirm, you will purchase this item.
    </h6>
    <input id="purchased_item" name="purchased_item" type="hidden" value="{{ item.name }}">
    {{ purchase_form.submit(class="btn btn-outline-success btn-block") }}
</form>